## Knapsack problem

In [5]:
import random
from prettytable import PrettyTable
import numpy as np
import math 

### Brute Force

In [8]:

n_items = 5
area = 50

prices = [random.randint(1, area) for _ in range(n_items)]
weights = [random.randint(1, area) for _ in range(n_items)]

tbl = PrettyTable()
tbl.field_names = ["ID","Prices", "Weight"]

for i in range(n_items):
    tbl.add_row([i, prices[i], weights[i]])

tbl.add_row(["---","------", "------"])
tbl.add_row(["Suma", sum(prices), sum(weights)])
print(tbl)



+------+--------+--------+
|  ID  | Prices | Weight |
+------+--------+--------+
|  0   |   10   |   29   |
|  1   |   7    |   42   |
|  2   |   28   |   47   |
|  3   |   29   |   35   |
|  4   |   44   |   31   |
| ---  | ------ | ------ |
| Suma |  118   |  184   |
+------+--------+--------+


In [10]:
def knapsack_brute_force(prices, weights, max_weight):
    n_products = len(prices)
    best_value = 0
    best_combination = []
    weights_of_b_comb = []

    for i in range(2 ** n_products):
        combination = []
        total_weight = 0
        total_value = 0

        for j in range(n_products):

            if (i >> j) & 1:
                combination.append(j)
                total_weight += weights[j]
                total_value += prices[j]

        if total_weight <= max_weight and total_value > best_value:
            best_value = total_value
            best_combination = combination
            
            weights_of_b_comb = weights

    return best_value, best_combination, weights_of_b_comb

best_value, best_combination, weights_of_best_combination = knapsack_brute_force(prices, weight, 200)
print("Maximální hodnota:", best_value)
print("Nejlepší kombinace produktů:", best_combination)
print("Váhy nejlepších produktů:", weights_of_best_combination)


Maximální hodnota: 118
Nejlepší kombinace produktů: [0, 1, 2, 3, 4]
Váhy nejlepších produktů: [17, 11, 4, 19, 32]


### Simulated annealing

In [14]:
def generate_starting_vector(weights, max_weight):
    starting_vector = []
    total_weight = 0

    for i in range(len(weights)*2):
        index = random.randint(0, n_items - 1)  # Vyber náhodný index produktu
        if index not in starting_vector:
            new_weight = total_weight + weights[index]
            if new_weight <= max_weight:
                starting_vector.append(index)
                total_weight = new_weight
    return starting_vector

def count_total_price(current_solution):
    total_price = 0
    for index in current_solution:
        total_price += prices[index]
    return total_price

def count_total_weight(current_solution):
    total_weight = 0
    for index in current_solution:
        total_weight += weights[index]
    return total_weight

def generate_neighbors(current_solution, n_neighbors, area):
    neighbors = []
    min_value = -area
    max_value = area
    step_10_percent = 0.1 * (max_value - min_value)
    sigma = step_10_percent/2
    for i in range(n_neighbors):
        neighbor = current_solution + np.random.normal(0, sigma, len(current_solution))
        while not all((neighbor >= -(area)) & (neighbor <= area)):
            neighbor = np.clip(neighbor, min_value, max_value)
        neighbors.append(neighbor)
    return neighbors


def knapsack_simulated_annealing(max_weight, prices, weights, n_neighbours, area, max_temp, min_temp, cooling_decr):
    n_products = len(prices)
    starting_vector = generate_starting_vector(weights, max_weight)
    current_solution = starting_vector #indexes of products

    total_price = count_total_price(current_solution)
    total_weight = count_total_weight(current_solution)

    current_fitness = total_price
    history = [current_fitness]
    T = max_temp

    while T > min_temp:
        neighbours = generate_neighbors(current_solution, n_neighbours, area)

        for neighbour_solution in neighbours:
            neighbour_fitness = count_total_price(neighbour_solution) 
            delta_f = current_fitness - neighbour_fitness

            if(delta_f < 0):
                current_fitness = neighbour_fitness
                current_solution = neighbour_solution

            elif(neighbour_fitness > current_fitness):
                current_fitness = neighbour_fitness
                current_solution = neighbour_solution

            elif(random.random() < math.exp(-delta_f/T)): #volani metropolise
                current_fitness = neighbour_fitness
                current_solution = neighbour_solution
                
            history.append(current_fitness) # pro 10 000 hodnot v historii
        #history.append(current_fitness)
        T = max_temp*cooling_decr**i
        if(T < min_temp):
            T = min_temp
    return history

In [2]:
array = [21, 12, 43, 39, 21]

In [3]:
cost_function(array)

136

In [15]:
generate_starting_vector(array,40)

[4, 1]